In [1]:
import os
import cv2
import zipfile
import tempfile
import shutil
import numpy as np
from pathlib import Path
import time
import sys
import json
import datetime

# Importar tu clase ObjectCounter desde Tracker3.py
try:
    from Tracker3 import ObjectCounter
except ImportError:
    print("No se pudo importar ObjectCounter desde Tracker3.py")
    print("Utilizando una versión simulada para continuar...")
    
    # Versión simplificada en caso de que falle la importación
    class ObjectCounter:
        def __init__(self, nombre_archivo, region, model="yolo11s.pt", classes=[0,2,3,5,7], 
                    show_in=True, show_out=True, line_width=2):
            self.nombre_archivo = nombre_archivo
            self.region = region
            self.model = model
            self.classes = classes
            self.show_in = show_in
            self.show_out = show_out
            self.line_width = line_width
            self.in_count = 0
            self.out_count = 0
            print(f"Inicializando modelo {model} para contar clases {classes}")
            
        def count(self, frame):
            # Dibujar la línea entre los dos puntos
            cv2.line(frame, self.region[0], self.region[1], (0, 0, 255), self.line_width)
            
            # Mostrar contadores simulados
            cv2.putText(frame, f"IN: {self.in_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
            cv2.putText(frame, f"OUT: {self.out_count}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
            
            return frame

# Función para formatear tamaño de archivos
def formatear_tamaño(bytes, sufijo="B"):
    factor = 1024
    for unidad in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f"{bytes:.2f} {unidad}{sufijo}"
        bytes /= factor
    return f"{bytes:.2f} E{sufijo}"

# Clase para gestionar el registro de videos procesados
class RegistroVideos:
    def __init__(self, ruta_registro=None):
        # Si no se especifica una ruta, usar el directorio del usuario
        if ruta_registro is None:
            self.ruta_registro = os.path.expanduser("~/videos_procesados.json")
        else:
            self.ruta_registro = ruta_registro
        
        # Inicializar o cargar el registro
        self.registro = self._cargar_registro()
    
    def _cargar_registro(self):
        # Crear el archivo si no existe
        if not os.path.exists(self.ruta_registro):
            # Crear un registro vacío
            registro_inicial = {
                "videos_procesados": {},
                "ultima_actualizacion": datetime.datetime.now().isoformat()
            }
            
            # Guardar el registro inicial
            with open(self.ruta_registro, 'w') as f:
                json.dump(registro_inicial, f, indent=2)
                
            return registro_inicial
        
        # Cargar el registro existente
        try:
            with open(self.ruta_registro, 'r') as f:
                return json.load(f)
        except Exception as e:
            print(f"Error al cargar el registro: {str(e)}")
            return {"videos_procesados": {}, "ultima_actualizacion": datetime.datetime.now().isoformat()}
    
    def esta_procesado(self, ruta_zip, nombre_video):
        # Crear un identificador único para el video
        id_video = f"{ruta_zip}:{nombre_video}"
        
        # Verificar si el video está en el registro
        return id_video in self.registro["videos_procesados"]
    
    def marcar_como_procesado(self, ruta_zip, nombre_video, resultado="completado"):
        # Crear un identificador único para el video
        id_video = f"{ruta_zip}:{nombre_video}"
        
        # Agregar el video al registro
        self.registro["videos_procesados"][id_video] = {
            "ruta_zip": ruta_zip,
            "nombre_video": nombre_video,
            "fecha_procesamiento": datetime.datetime.now().isoformat(),
            "resultado": resultado
        }
        
        # Actualizar la fecha de última actualización
        self.registro["ultima_actualizacion"] = datetime.datetime.now().isoformat()
        
        # Guardar el registro actualizado
        self._guardar_registro()
    
    def _guardar_registro(self):
        try:
            with open(self.ruta_registro, 'w') as f:
                json.dump(self.registro, f, indent=2)
        except Exception as e:
            print(f"Error al guardar el registro: {str(e)}")
    
    def obtener_estadisticas(self):
        # Obtener estadísticas del registro
        total_videos = len(self.registro["videos_procesados"])
        completados = sum(1 for v in self.registro["videos_procesados"].values() if v["resultado"] == "completado")
        con_errores = sum(1 for v in self.registro["videos_procesados"].values() if v["resultado"] == "error")
        
        return {
            "total_videos": total_videos,
            "completados": completados,
            "con_errores": con_errores,
            "ultima_actualizacion": self.registro["ultima_actualizacion"]
        }
    
    def obtener_videos_procesados_por_zip(self, ruta_zip):
        # Obtener los videos procesados para un ZIP específico
        videos_procesados = []
        
        for id_video, info in self.registro["videos_procesados"].items():
            if info["ruta_zip"] == ruta_zip:
                videos_procesados.append(info["nombre_video"])
        
        return videos_procesados

# Función para listar videos dentro de un archivo ZIP
def listar_videos_en_zip(ruta_zip):
    videos = []
    extensiones_video = ['.mp4', '.avi', '.mov', '.mkv', '.wmv', '.flv']
    
    try:
        with zipfile.ZipFile(ruta_zip, 'r') as zip_ref:
            for archivo in zip_ref.namelist():
                if any(archivo.lower().endswith(ext) for ext in extensiones_video):
                    videos.append(archivo)
    except Exception as e:
        print(f"Error al leer {ruta_zip}: {str(e)}")
    
    return videos

# Función para encontrar archivos ZIP
def encontrar_archivos_zip(directorio_base):
    archivos_zip = []
    
    for carpeta_actual, _, archivos in os.walk(directorio_base):
        for archivo in archivos:
            if archivo.lower().endswith('.zip'):
                ruta_completa = os.path.join(carpeta_actual, archivo)
                archivos_zip.append(ruta_completa)
    
    return archivos_zip

# Función para extraer un solo video del ZIP a un archivo temporal
def extraer_video_temporal(ruta_zip, nombre_video):
    try:
        # Crear un directorio temporal
        temp_dir = tempfile.mkdtemp()
        
        # Extraer solo el archivo de video específico
        with zipfile.ZipFile(ruta_zip, 'r') as zip_ref:
            zip_ref.extract(nombre_video, temp_dir)
        
        # Ruta completa al archivo extraído
        ruta_temp = os.path.join(temp_dir, nombre_video)
        
        return temp_dir, ruta_temp
    except Exception as e:
        print(f"Error al extraer {nombre_video}: {str(e)}")
        return None, None

# Declarar las variables globales al inicio del script
imagen = None
puntos = []
video_iniciado = False

# Función para procesar un video usando tu código original
def procesar_video(ruta_video):
    global imagen, puntos, video_iniciado
    
    # Reiniciar las variables para cada nuevo video
    puntos = []
    video_iniciado = False

    def seleccionar_puntos(event, x, y, flags, param):
        global imagen, puntos, video_iniciado

        if event == cv2.EVENT_LBUTTONDOWN:  # Si se hace clic izquierdo
            puntos.append((x, y))  # Agregar el punto a la lista

            # Dibujar el punto en la imagen
            cv2.circle(imagen, (x, y), 5, (0, 255, 0), -1)  # Poner un círculo verde en el punto

            if len(puntos) == 2:  # Si ya se han seleccionado 2 puntos
                # Dibujar la línea entre los dos puntos
                cv2.line(imagen, puntos[0], puntos[1], (0, 0, 255), 2)  # Línea roja
                
                cv2.imshow("nombrepestania", imagen)

    # Open the video file
    cap = cv2.VideoCapture(ruta_video)

    # Leer el primer frame del video
    ret, imagen = cap.read()
    
    if not ret:
        print("No se pudo leer el primer frame del video")
        return False
    
    # Redimensionar como en tu código original
    imagen = cv2.resize(imagen, (1020, 500))

    cv2.imshow("nombrepestania", imagen)
    cv2.setMouseCallback("nombrepestania", seleccionar_puntos)

    # Esperar hasta que el usuario seleccione los dos puntos
    while len(puntos) < 2:
        # Check if the window was closed with the 'x' button
        if cv2.getWindowProperty("nombrepestania", cv2.WND_PROP_VISIBLE) < 1:
            cap.release()
            cv2.destroyAllWindows()
            return False
        key = cv2.waitKey(1)
        if key == 27:  # ESC key to exit
            cap.release()
            cv2.destroyAllWindows()
            return False

    # Reproducir el video después de seleccionar los dos puntos
    count = 0
    region_points = [puntos[0], puntos[1]]

    # Initialize the object counter with our original implementation
    try:
        # Try to use your advanced ObjectCounter from Tracker3.py
        counter = ObjectCounter(
            nombre_archivo=os.path.basename(ruta_video),
            region=region_points,  # Pass region points
            model="yolo11s.pt",
            classes=[0,2,3,5,7],  # Classes for detection
            show_in=True,  # Display in counts
            show_out=True,  # Display out counts
            line_width=2,  # Adjust line width for display
        )
    except Exception as e:
        print(f"Error al inicializar ObjectCounter avanzado: {str(e)}")
        print("Utilizando contador simple...")
        # Fallback to a simpler implementation if the advanced one fails
        counter = ObjectCounter(
            nombre_archivo=ruta_video,
            region=region_points,
            model="yolo11s.pt",
            classes=[0,2,3,5,7],
            show_in=True,
            show_out=True,
            line_width=2,
        )

    # Reiniciar el video para procesarlo desde el principio
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

    while True:
        ret, imagen = cap.read()

        # Si no se puede leer el frame, salir del bucle
        if not ret:
            break

        count += 1
        if count % 2 != 0:  # Skip odd frames como en tu código original
            continue

        imagen = cv2.resize(imagen, (1020, 500))

        # Process the frame with the object counter
        try:
            imagen = counter.count(imagen)
        except Exception as e:
            print(f"Error al procesar frame: {str(e)}")
            # Dibujar la línea manualmente si falla el contador
            cv2.line(imagen, puntos[0], puntos[1], (0, 0, 255), 2)

        # Mostrar el frame actual
        cv2.imshow("nombrepestania", imagen)

        # Salir si presionas la tecla 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the video capture object and close the display window
    cap.release()
    cv2.destroyAllWindows()
    print("Procesamiento completado.")
    
    return True  # Indicar que el procesamiento se completó correctamente

# Función principal simplificada
def procesar_archivos():
    print("\n" + "="*50)
    print("PROCESADOR DE VIDEOS EN ARCHIVOS ZIP")
    print("="*50)
    
    # Inicializar el registro de videos procesados
    registro = RegistroVideos()
    
    # Mostrar estadísticas básicas
    estadisticas = registro.obtener_estadisticas()
    print(f"Videos procesados: {estadisticas['total_videos']}")
    
    # Solicitar únicamente la ruta al disco duro
    directorio_base = input("Ingrese la ruta al disco duro con los videos ZIP: ").strip()
    
    if not os.path.exists(directorio_base):
        print(f"La ruta {directorio_base} no existe.")
        return
    
    print("\nBuscando archivos ZIP...")
    archivos_zip = encontrar_archivos_zip(directorio_base)
    
    if not archivos_zip:
        print("No se encontraron archivos ZIP en la ruta especificada.")
        return
    
    print(f"Se encontraron {len(archivos_zip)} archivos ZIP.")
    
    # Procesar cada archivo ZIP automáticamente
    for i, ruta_zip in enumerate(archivos_zip):
        nombre_zip = os.path.basename(ruta_zip)
        print(f"\n[{i+1}/{len(archivos_zip)}] Procesando archivo: {nombre_zip}")
        
        # Listar videos en el ZIP
        videos = listar_videos_en_zip(ruta_zip)
        videos_procesados = registro.obtener_videos_procesados_por_zip(ruta_zip)
        videos_pendientes = [v for v in videos if v not in videos_procesados]
        
        print(f"Videos encontrados: {len(videos)}")
        print(f"Videos pendientes: {len(videos_pendientes)}")
        
        if not videos_pendientes:
            print("Todos los videos de este ZIP ya fueron procesados. Pasando al siguiente...")
            continue
        
        # Procesar videos pendientes uno por uno
        for j, video_nombre in enumerate(videos_pendientes):
            print(f"\n[{j+1}/{len(videos_pendientes)}] Procesando: {video_nombre}")
            
            # Extraer y procesar el video
            temp_dir, ruta_temp = extraer_video_temporal(ruta_zip, video_nombre)
            
            if not temp_dir or not ruta_temp:
                print(f"No se pudo extraer {video_nombre}. Continuando con el siguiente...")
                registro.marcar_como_procesado(ruta_zip, video_nombre, "error")
                continue
            
            try:
                # Procesar el video usando tu código original
                exito = procesar_video(ruta_temp)
                
                # Actualizar el registro
                if exito:
                    registro.marcar_como_procesado(ruta_zip, video_nombre, "completado")
                    print(f"Video procesado con éxito.")
                else:
                    registro.marcar_como_procesado(ruta_zip, video_nombre, "error")
                    print(f"Error al procesar el video.")
            finally:
                # Limpiar archivos temporales
                try:
                    shutil.rmtree(temp_dir)
                    print(f"Archivos temporales eliminados.")
                except Exception as e:
                    print(f"Error al eliminar archivos temporales: {str(e)}")

# Ejecutar el programa
if __name__ == "__main__":
    try:
        procesar_archivos()
    except KeyboardInterrupt:
        print("\nPrograma interrumpido por el usuario.")
    except Exception as e:
        print(f"\nError inesperado: {str(e)}")
    finally:
        print("\nPrograma finalizado.")


PROCESADOR DE VIDEOS EN ARCHIVOS ZIP
Videos procesados: 3

Buscando archivos ZIP...
Se encontraron 19 archivos ZIP.

[1/19] Procesando archivo: Enero 21-25.zip
Videos encontrados: 165
Videos pendientes: 162

[1/162] Procesando: 20210121/_0_1_1AB3F5D663EB415AAA3F7E02E0B5C456_/20210121_20210121221536_20210122071500_081536.mp4


2025-03-19 16:33:40.228 python[13958:10636019] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-19 16:33:40.228 python[13958:10636019] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Ultralytics Solutions: ✅ {'region': [(394, 41), (1002, 482)], 'show_in': True, 'show_out': True, 'colormap': None, 'up_angle': 145.0, 'down_angle': 90, 'kpts': [6, 8, 10], 'analytics_type': 'line', 'json_file': None, 'records': 5, 'model': 'yolo11s.pt', 'classes': [0, 2, 3, 5, 7], 'line_width': 2}
CSV file created: 20210121_20210121221536_20210122071500_081536.mp4_vehicle_count_data_.csv with headers.

0: 320x640 4 cars, 147.0ms
Speed: 3.5ms preprocess, 147.0ms inference, 10.9ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 cars, 97.5ms
Speed: 2.5ms preprocess, 97.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 cars, 93.3ms
Speed: 1.8ms preprocess, 93.3ms inference, 0.9ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 cars, 81.7ms
Speed: 1.8ms preprocess, 81.7ms inference, 0.9ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 cars, 81.3ms
Speed: 1.4ms preprocess, 81.3ms inference, 0.5ms postprocess per imag